In [ ]:
import torch
from torch import nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from torch.optim import AdamW
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2, fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.ops import nms
from torchvision.io import read_image
import torchvision.transforms.v2 as T
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import math
import utils1
from utils1 import * 
from evaluation import evaluate_with_results
from datasets import *
import torchvision.transforms as transforms
from visualisation import *

In [ ]:
# Define the model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.25)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 256)
        self.dropout2 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, 43)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = self.pool(x)
        x = self.dropout(x)
        x = torch.relu(self.conv3(x))
        x = torch.relu(self.conv4(x))
        x = self.pool(x)
        x = self.dropout(x)
        x = x.view(-1, 64 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        return x

In [ ]:
#Define Color Ranges
colors = {
    "red": (([0, 50, 50], [15, 255, 255]), ([165, 50, 50], [180, 255, 255])),
    "blue": (([90, 50, 50], [130, 255, 255]), None),
    "yellow": (([10, 50, 50], [30, 255, 255]), None),
}

penalty = {
    "red": 5,
    "blue": 7,
    "yellow": 50,
}

def get_viable_classes(image):
    mask = np.zeros((30, 30, 3), dtype=np.uint8)
    cv2.circle(mask, (15, 15), 15, (1, 1, 1), thickness=-1)
    masked_img = mask * image

    image_hsv = cv2.cvtColor(masked_img, cv2.COLOR_BGR2HSV)

    innerMask = np.zeros((30, 30, 1), dtype=np.uint8)
    cv2.rectangle(innerMask, (5, 5), (25, 25), (1), -1)
    
    #special case yellow
    max = 0
    dominantColor = ""
    for color in colors:
        colorMask = cv2.inRange(image_hsv, np.array(colors[color][0][0]), np.array(colors[color][0][1]))
        maskYellow = np.zeros((30, 30, 1), dtype=np.uint8)
        cv2.rectangle(maskYellow, (10,10), (20,20), (1), -1)
        colorMask = cv2.multiply(colorMask, maskYellow)
        if colorMask.sum() > max and (colorMask.sum()/255) >= 50:
            max = colorMask.sum()
            dominantColor = color
    if dominantColor == "yellow":
        return (color_map["yellow"], penalty["yellow"], dominantColor)
    # if dominantColor == "red":
    #     return (color_map["red"], 50, dominantColor)
    

    max = 0
    dominantColor = ""
    for color in colors:
        if color == "yellow":
            continue
        colorMask = cv2.inRange(image_hsv, np.array(colors[color][0][0]), np.array(colors[color][0][1]))
        if colors[color][1]: 
            colorMask = cv2.add(colorMask, cv2.inRange(image_hsv, np.array(colors[color][1][0]), np.array(colors[color][1][1])))
        if color == "blue":
            colorMask = cv2.multiply(colorMask, innerMask)
        if colorMask.sum() > max and (colorMask.sum()/255) >= 225:
            max = colorMask.sum()
            dominantColor = color
    if dominantColor == "":
        return (list(range(1, 43)), 0, dominantColor)
    return (color_map[dominantColor], penalty[dominantColor], dominantColor)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

#Settings
images_path_GTSDB = "GTSDBDataset"
annotaions_path_GTSDB = "GTSDBDataset/gt.txt"
localization_model_path = "Models/modelGTSDB_WithoutClasses19.pth"
classification_model_path = "Models/classification_new_30.pth"
batch_size = 4
nms_threshold = 0.7

In [ ]:
localization_model = torch.load(localization_model_path)
classification_model = torch.load(classification_model_path)
localization_model.to(device)
classification_model.to(device)
localization_model.eval()
classification_model.eval()
pass

In [ ]:
dataset_GTSDB = GTSDBDataset(True, images_path_GTSDB, annotaions_path_GTSDB)
dataloader_GTSDB = DataLoader(dataset_GTSDB, batch_size=batch_size, pin_memory=True, shuffle=False, collate_fn=collate_fn)

test_indices = list(range(600, 900))
test_dataset = Subset(dataset_GTSDB, test_indices)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, pin_memory=True, shuffle=False, collate_fn=collate_fn)

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
actual = {}
predictions = {}
idx = 0

for data in tqdm(test_dataloader):
    for i in range(batch_size):
        image = data[0][i].to(device)
        image_to_show = cv2.imread(data[2][i])

        outputs = localization_model([image])
        boxes = outputs[0]["boxes"].detach().cpu()
        scores = outputs[0]["scores"].detach().cpu()
        labels = outputs[0]["labels"].detach().cpu()
        
        to_keep = nms(boxes, scores, nms_threshold)
        boxes = boxes[to_keep]
        scores = scores[to_keep]

        images = extract_signs(image_to_show, boxes)
        predictions[idx] = []
        actual[idx] = []
        visualColors = {}
        for i2, box in enumerate(data[1][i]["boxes"]):
            sign = Sign(box[2], box[3], box[0], box[1], class_map[data[1][i]["labels"][i2].item()])
            actual[idx].append(sign)
        for i in range(len(images)):
            image = images[i]
            image = cv2.resize(image, (30, 30))
            viable_classes = get_viable_classes(image)
            image = transform(image)
            image = image.to(device)
            output = classification_model(image)
            for a in range(len(output[0])):
                if a not in viable_classes[0]:
                    output[0][a] -= viable_classes[1]
            _, predicted = torch.max(output.data, 1)
            box = boxes[i]
            sign = Sign(box[2], box[3], box[0], box[1], class_map[predicted.item()])
            predictions[idx].append(sign)
            visualColors[i] = viable_classes[2]
        #show_image_with_signs(image_to_show, predictions[idx], visualColors)
        idx += 1

In [ ]:
evaluate_with_results(actual, predictions)